# In this notebook we benchmark the performance of Sinkhon vs damped Newton for different distance functions.

In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import scipy as scp
import pylab as pyl

import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)

%matplotlib inline
%load_ext autoreload
%autoreload 

In [ ]:
relative_path_to_new_folder = "../Images"
os.makedirs(relative_path_to_new_folder, exist_ok=True)
if not os.path.isdir('../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images'):
    os.makedirs('../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images')

In [ ]:
def p_norms(x,y,p):
   
   if p == 1:
      
      return  abs(np.sum(x,0)[:,None]-np.sum(y,0)[None,:])  
   
   elif p == 2:
      return  np.sum( x**2,0 )[:,None] + np.sum( y**2,0 )[None,:] - 2*x.transpose().dot(y)
   
   elif p == 4:
    return np.sum( x**4,0 )[:,None] + np.sum( y**4,0 )[None,:] + 4*(x**3).transpose().dot(y) - 4*x.transpose().dot(y**3) + 6*(x**2).transpose().dot(y**2)

   elif p == 'inf':
      return np.maximum(abs(x[0,:][:,None]-y[0,:][None,:]),abs(x[1,:][:,None] - y[1:][None:]))




"""To compute distance matrix"""
def distmat(x,y):
    return np.sum( x**2,0 )[:,None] + np.sum( y**2,0 )[None,:] - 2*x.transpose().dot(y)

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP(u,K,v):
    return u[:,None]*K*v[None,:]

def plotp(x, col,plt, scale=200, edgecolors="k"):
  return plt.scatter( x[0,:], x[1,:], s=scale, edgecolors=edgecolors,  c=col, cmap='plasma', linewidths=2 )

In [ ]:
import computational_OT

In [ ]:
def spectral_decomposition(mat):
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort(eig)
    eig = eig[sorting_indices]
    v   = v[:, sorting_indices]
    
    print( "List of smallest eigenvalues: ", eig[:10])
    print( "List of largest  eigenvalues: ", eig[-10:])

    return eig,v

In [ ]:
def build_preconditioners( num_eigs,modified_Hessian, N, ansatz=True ):
    # Diagonalize
    eigenvalues, eigenvectors = np.linalg.eigh( modified_Hessian )
    sorting_indices = np.argsort( eigenvalues )
    eigenvalues  = eigenvalues[sorting_indices]
    eigenvectors = eigenvectors[:, sorting_indices]
    # Form null vector
    if not ansatz:
        null_vector = eigenvectors[:, 0]
    else:
        null_vector = np.hstack( (np.ones(N[0]), -np.ones(N[1])) )
        norm = np.sqrt( N[0] + N[1] )
        null_vector = null_vector/norm
    # Form other vectors (only 13)
    n,m = eigenvectors.shape
    indices=[]
    for i in range(num_eigs//2):
        indices.append(m-i-2)
        indices.append(i+1)
    if num_eigs%2!=0:
        indices.append(m-1-(num_eigs//2))
   
    precond_vectors = eigenvectors[:, indices ]
    precond_vectors = []
    for index in indices:
        precond_vectors.append( eigenvectors[:,index] )
    #
    return null_vector, precond_vectors

In [ ]:
N = [ 1000,1200 ]
epsilons = [ 0.1,0.09,0.05,0.03 ] 


In [ ]:
x = np.random.rand( 2,N[0] ) - 0.5
theta = 2*np.pi*np.random.rand( 1,N[1] )
r = 0.8+.2*np.random.rand( 1,N[1] )
y = np.vstack( ( r*np.cos( theta ),r*np.sin( theta ) ) )

# I. L-1 norm

###  Sinkhorn

In [ ]:
# Sinkhorn
print("Sinkhorn.... ")
SinkhornP=[]
results_Sinkhorn=[]
times_Sinkhorn=[]
for eps in epsilons:

  
  #Cost matrix
  C = p_norms(x,y,2)
  
  # a and b
  a = normalize(np.ones(N[0]))
  a = a.reshape(a.shape[0],-1)
  b = normalize(np.ones(N[1]))
  b = b.reshape(b.shape[0],-1)



  #Kernel
  K = np.exp(-C/eps)


  print( "Doing for (",N[0],N[1],") and epsilon = " +str(eps)+ " ." )
  print( " |- Iterating" )

  #Inflating
  u = a
  v = b

  start = time.time()
  Optimizer = computational_OT.Sinkhorn( K,a,b,u,v,eps)
  out = Optimizer._update(maxiter=10000)
  results_Sinkhorn.append( out )
  end = time.time()
  times_Sinkhorn.append( end-start )
  print( " |- Computing P" )
  print( "" )
  SinkhornP.append(GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)))
  

In [ ]:
plt.figure( figsize = (20,7) )

plt.subplot(2,1,1),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len(results_Sinkhorn) ):
  error=np.asarray( results_Sinkhorn[i]['error_a'] )+np.asarray( results_Sinkhorn[i]['error_b'] )
  plt.plot( error,label='Sinkhorn for $\epsilon=$'+ str(epsilons[i]), linewidth = 2 )
plt.yscale( 'log' )
plt.legend()

plt.savefig("../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images/ConvergenceSinkhornvaryingepsilonL1.png")
plt.show()

###  Damped Newton


In [ ]:
rho = 0.95
c = 0.05
DampedNewtonP = []
results_DampedNewton  = []
times_DampedNewton    = []
Hessians_DampedNewton = []

for eps in epsilons:
    # Line Search
    print( "Damped Newton for epsilon = "+str(eps)+":" )    
    #Cost matrix
    C = p_norms(x,y,1)

    # a and b
    a = normalize( np.ones( N[0] ) )
    a = a.reshape( a.shape[0],-1 )
    b = normalize( np.ones( N[1] ) )
    b = b.reshape( b.shape[0],-1 )

    #Kernel
    K = np.exp(-C/eps)
    f,g = a,b

    print( "Doing for (",N[0],N[1],")." )
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.DampedNewton( K,a,b,f,g,eps,rho,c )
    out = Optimizer._update( maxiter=50 )
    results_DampedNewton.append( out )
    end = time.time()
    times_DampedNewton.append( end-start )
    print( " |- Computing P" )
    DampedNewtonP.append( GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)) )
    print( " |- Recording (unstabilized) Hessian \n" )

    mat  = -eps*Optimizer.Hessian
    diag = 1/np.sqrt( np.vstack( (a,b) ) ).flatten()
    mat = diag[:,None]*mat*diag[None,:]
    Hessians_DampedNewton.append( mat )
    

In [ ]:
eigs = []
eigvecs = []
for i in range( len(epsilons) ) :
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon="+str(eps) )
    ev = spectral_decomposition( Hessians_DampedNewton[i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")


In [ ]:
fig,ax=plt.subplots(figsize=(5,12), nrows=len(epsilons), ncols=1, sharey=True)
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
plt.subplots_adjust( wspace=0,hspace=0 )
plt.savefig("../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images/eigenhistunstabilizedL1.png")
plt.show()

### Damped Newton with preconditioning

In [ ]:
num_eigs = 10
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_DampedNewton[-1],N, ansatz=False )

In [ ]:
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians = []
DampedNewtonP = []
results_DampedNewton  = []
times_DampedNewton    = []

#epsilons = [ 0.3 ]
f, g = None, None
for eps in epsilons:
    # Line Search
    print( "Damped Newton for epsilon="+str(eps)+":" )    
    # Cost matrix
    C = p_norms(x,y,1)

    # a and b
    a = normalize( np.ones(N[0]) )
    a = a.reshape( a.shape[0],-1 )
    b = normalize( np.ones(N[1]) )
    b = b.reshape( b.shape[0],-1 )

    #Kernel
    K = np.exp(-C/eps)

    if (f is None) or (g is None): 
        f,g = a,b

    print( "Doing for (",N[0],N[1],")." )
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.DampedNewton_With_Preconditioner( K,a,b,f,g,eps,rho,c,null_vector,precond_vectors[:] )
    out = Optimizer._update( maxiter=50, iterative_inversion=30, version=None,debug=False,optType='cg' )
    results_DampedNewton.append( out )
    end = time.time()
    times_DampedNewton.append(end-start)
    print( " |- Computing P" )

    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
        # f = f.reshape( f.shape[0], -1)
        # g = g.reshape( g.shape[0], -1)
    
    DampedNewtonP.append( GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)) )
    final_modified_Hessians.append( Optimizer.modified_Hessian )


In [ ]:
plt.figure( figsize = (20,7) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range( len(results_DampedNewton) ):
  error = np.asarray( results_DampedNewton[i]['error_a'] )+np.asarray( results_DampedNewton[i]['error_b'] )
  plt.plot( error,label='Damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2 )

plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images/ErrorLinesearchNewton_final_cgL1.png" )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

# II. L-2 norm

## Sinkhorn

In [ ]:
# Sinkhorn
print("Sinkhorn.... ")
SinkhornP=[]
results_Sinkhorn=[]
times_Sinkhorn=[]
for eps in epsilons:

  
  #Cost matrix
  C = p_norms(x,y,2)
  
  # a and b
  a = normalize(np.ones(N[0]))
  a = a.reshape(a.shape[0],-1)
  b = normalize(np.ones(N[1]))
  b = b.reshape(b.shape[0],-1)



  #Kernel
  K = np.exp(-C/eps)


  print( "Doing for (",N[0],N[1],") and epsilon = " +str(eps)+ " ." )
  print( " |- Iterating" )

  #Inflating
  u = a
  v = b

  start = time.time()
  Optimizer = computational_OT.Sinkhorn( K,a,b,u,v,eps)
  out = Optimizer._update(maxiter=10000)
  results_Sinkhorn.append( out )
  end = time.time()
  times_Sinkhorn.append( end-start )
  print( " |- Computing P" )
  print( "" )
  SinkhornP.append(GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)))
  

In [ ]:
plt.figure( figsize = (20,7) )

plt.subplot(2,1,1),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len(results_Sinkhorn) ):
  error=np.asarray( results_Sinkhorn[i]['error_a'] )+np.asarray( results_Sinkhorn[i]['error_b'] )
  plt.plot( error,label='Sinkhorn for $\epsilon=$'+ str(epsilons[i]), linewidth = 2 )
plt.yscale( 'log' )
plt.legend()

plt.savefig("../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images/ConvergenceSinkhornvaryingepsilonL2.png")
plt.show()

## Damped Newton without preconditioning


In [ ]:
rho = 0.95
c = 0.05
DampedNewtonP = []
results_DampedNewton  = []
times_DampedNewton    = []
Hessians_DampedNewton = []

for eps in epsilons:
    # Line Search
    print( "Damped Newton for epsilon = "+str(eps)+":" )    
    #Cost matrix
    C = p_norms(x,y,2)

    # a and b
    a = normalize( np.ones( N[0] ) )
    a = a.reshape( a.shape[0],-1 )
    b = normalize( np.ones( N[1] ) )
    b = b.reshape( b.shape[0],-1 )

    #Kernel
    K = np.exp(-C/eps)
    f,g = a,b

    print( "Doing for (",N[0],N[1],")." )
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.DampedNewton( K,a,b,f,g,eps,rho,c )
    out = Optimizer._update( maxiter=50 )
    results_DampedNewton.append( out )
    end = time.time()
    times_DampedNewton.append( end-start )
    print( " |- Computing P" )
    DampedNewtonP.append( GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)) )
    print( " |- Recording (unstabilized) Hessian \n" )

    mat  = -eps*Optimizer.Hessian
    diag = 1/np.sqrt( np.vstack( (a,b) ) ).flatten()
    mat = diag[:,None]*mat*diag[None,:]
    Hessians_DampedNewton.append( mat )
    

In [ ]:
eigs = []
eigvecs = []
for i in range( len(epsilons) ) :
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon="+str(eps) )
    ev = spectral_decomposition( Hessians_DampedNewton[i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")


In [ ]:
fig,ax = plt.subplots( figsize=(24,3),nrows=1, ncols=len(epsilons), sharey=True )
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
plt.subplots_adjust( wspace=0,hspace=0 )
plt.savefig("../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images/eigenhistunstabilizedL2.png")
plt.show()

In [ ]:
num_eigs = 10
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_DampedNewton[-1],N, ansatz=False )

## Damped Newton with preconditioning

In [ ]:
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians = []
DampedNewtonP = []
results_DampedNewton  = []
times_DampedNewton    = []

#epsilons = [ 0.3 ]
f, g = None, None
for eps in epsilons:
    # Line Search
    print( "Damped Newton for epsilon="+str(eps)+":" )    
    # Cost matrix
    C = p_norms(x,y,2)

    # a and b
    a = normalize( np.ones(N[0]) )
    a = a.reshape( a.shape[0],-1 )
    b = normalize( np.ones(N[1]) )
    b = b.reshape( b.shape[0],-1 )

    #Kernel
    K = np.exp(-C/eps)

    if (f is None) or (g is None): 
        f,g = a,b

    print( "Doing for (",N[0],N[1],")." )
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.DampedNewton_With_Preconditioner( K,a,b,f,g,eps,rho,c,null_vector,precond_vectors[:] )
    out = Optimizer._update( maxiter=50, iterative_inversion=30, version=None,debug=False,optType='cg' )
    results_DampedNewton.append( out )
    end = time.time()
    times_DampedNewton.append(end-start)
    print( " |- Computing P" )

    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
        # f = f.reshape( f.shape[0], -1)
        # g = g.reshape( g.shape[0], -1)
    
    DampedNewtonP.append( GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)) )
    final_modified_Hessians.append( Optimizer.modified_Hessian )


In [ ]:
plt.figure( figsize = (20,7) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range( len(results_DampedNewton) ):
  error = np.asarray( results_DampedNewton[i]['error_a'] )+np.asarray( results_DampedNewton[i]['error_b'] )
  plt.plot( error,label='Damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2 )

plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images/ErrorLinesearchNewton_final_cgL2.png" )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

# III. L4-norm

## Sinkhorn

In [ ]:
print("Sinkhorn.... ")
SinkhornP=[]
results_Sinkhorn=[]
times_Sinkhorn=[]
for eps in epsilons:

  
  #Cost matrix
  C = p_norms(x,y,4)
  
  # a and b
  a = normalize(np.ones(N[0]))
  a = a.reshape(a.shape[0],-1)
  b = normalize(np.ones(N[1]))
  b = b.reshape(b.shape[0],-1)



  #Kernel
  K = np.exp(-C/eps)


  print( "Doing for (",N[0],N[1],") and epsilon = " +str(eps)+ " ." )
  print( " |- Iterating" )

  #Inflating
  u = a
  v = b

  start = time.time()
  Optimizer = computational_OT.Sinkhorn( K,a,b,u,v,eps)
  out = Optimizer._update(maxiter=10000)
  results_Sinkhorn.append( out )
  end = time.time()
  times_Sinkhorn.append( end-start )
  print( " |- Computing P" )
  print( "" )
  SinkhornP.append(GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)))
  

In [ ]:
plt.figure( figsize = (20,7) )

plt.subplot(2,1,1),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len(results_Sinkhorn) ):
  error=np.asarray( results_Sinkhorn[i]['error_a'] )+np.asarray( results_Sinkhorn[i]['error_b'] )
  plt.plot( error,label='Sinkhorn for $\epsilon=$'+ str(epsilons[i]), linewidth = 2 )
plt.yscale( 'log' )
plt.legend()

plt.savefig("../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images/ConvergenceSinkhornvaryingepsilonL4.png")
plt.show()

## Damped Newton without preconditioning

In [ ]:
rho = 0.95
c = 0.05
DampedNewtonP = []
results_DampedNewton  = []
times_DampedNewton    = []
Hessians_DampedNewton = []

for eps in epsilons:
    # Line Search
    print( "Damped Newton for epsilon = "+str(eps)+":" )    
    #Cost matrix
    C = p_norms(x,y,4)

    # a and b
    a = normalize( np.ones( N[0] ) )
    a = a.reshape( a.shape[0],-1 )
    b = normalize( np.ones( N[1] ) )
    b = b.reshape( b.shape[0],-1 )

    #Kernel
    K = np.exp(-C/eps)
    f,g = a,b

    print( "Doing for (",N[0],N[1],")." )
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.DampedNewton( K,a,b,f,g,eps,rho,c )
    out = Optimizer._update( maxiter=50 )
    results_DampedNewton.append( out )
    end = time.time()
    times_DampedNewton.append( end-start )
    print( " |- Computing P" )
    DampedNewtonP.append( GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)) )
    print( " |- Recording (unstabilized) Hessian \n" )

    mat  = -eps*Optimizer.Hessian
    diag = 1/np.sqrt( np.vstack( (a,b) ) ).flatten()
    mat = diag[:,None]*mat*diag[None,:]
    Hessians_DampedNewton.append( mat )
    

In [ ]:
eigs = []
eigvecs = []
for i in range( len(epsilons) ) :
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon="+str(eps) )
    ev = spectral_decomposition( Hessians_DampedNewton[i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")


In [ ]:
fig,ax = plt.subplots( figsize=(24,3),nrows=1, ncols=len(epsilons), sharey=True )
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    if not np.isnan(eigs[i]).any():
        ax[i].hist( eigs[i], 50 )
        ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
        ax[i].set_xlabel( "Eigenvalues" )
        ax[i].set_yscale( "log" )
    
    else:
        print("The eigenvalues for "+str(epsilons[i])+ " contains Nans.")
plt.subplots_adjust( wspace=0,hspace=0 )
plt.savefig("../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images/eigenhistunstabilizedL2.png")
plt.show()

In [ ]:
num_eigs = 10
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_DampedNewton[-1],N, ansatz=False )

## Damped Newton with preconditioning

In [ ]:
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians = []
DampedNewtonP = []
results_DampedNewton  = []
times_DampedNewton    = []

#epsilons = [ 0.3 ]
f, g = None, None
for eps in epsilons:
    # Line Search
    print( "Damped Newton for epsilon="+str(eps)+":" )    
    # Cost matrix
    C = p_norms(x,y,4)

    # a and b
    a = normalize( np.ones(N[0]) )
    a = a.reshape( a.shape[0],-1 )
    b = normalize( np.ones(N[1]) )
    b = b.reshape( b.shape[0],-1 )

    #Kernel
    K = np.exp(-C/eps)

    if (f is None) or (g is None): 
        f,g = a,b

    print( "Doing for (",N[0],N[1],")." )
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.DampedNewton_With_Preconditioner( K,a,b,f,g,eps,rho,c,null_vector,precond_vectors[:] )
    out = Optimizer._update( maxiter=50, iterative_inversion=30, version=None,debug=False,optType='cg' )
    results_DampedNewton.append( out )
    end = time.time()
    times_DampedNewton.append(end-start)
    print( " |- Computing P" )

    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
        # f = f.reshape( f.shape[0], -1)
        # g = g.reshape( g.shape[0], -1)
    
    DampedNewtonP.append( GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)) )
    final_modified_Hessians.append( Optimizer.modified_Hessian )


In [ ]:
plt.figure( figsize = (20,7) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range( len(results_DampedNewton) ):
  error = np.asarray( results_DampedNewton[i]['error_a'] )+np.asarray( results_DampedNewton[i]['error_b'] )
  plt.plot( error,label='Damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2 )

plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images/ErrorLinesearchNewton_final_cgL2.png" )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

# IV. L-infinity

## Sinkhorn

In [ ]:
print("Sinkhorn.... ")
SinkhornP=[]
results_Sinkhorn=[]
times_Sinkhorn=[]
for eps in epsilons:

  
  #Cost matrix
  C = p_norms(x,y,'inf')
  
  # a and b
  a = normalize(np.ones(N[0]))
  a = a.reshape(a.shape[0],-1)
  b = normalize(np.ones(N[1]))
  b = b.reshape(b.shape[0],-1)



  #Kernel
  K = np.exp(-C/eps)


  print( "Doing for (",N[0],N[1],") and epsilon = " +str(eps)+ " ." )
  print( " |- Iterating" )

  #Inflating
  u = a
  v = b

  start = time.time()
  Optimizer = computational_OT.Sinkhorn( K,a,b,u,v,eps)
  out = Optimizer._update(maxiter=10000)
  results_Sinkhorn.append( out )
  end = time.time()
  times_Sinkhorn.append( end-start )
  print( " |- Computing P" )
  print( "" )
  SinkhornP.append(GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)))
  

In [ ]:
plt.figure( figsize = (20,7) )

plt.subplot(2,1,1),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len(results_Sinkhorn) ):
  error=np.asarray( results_Sinkhorn[i]['error_a'] )+np.asarray( results_Sinkhorn[i]['error_b'] )
  plt.plot( error,label='Sinkhorn for $\epsilon=$'+ str(epsilons[i]), linewidth = 2 )
plt.yscale( 'log' )
plt.legend()

plt.savefig("../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images/ConvergenceSinkhornvaryingepsilonL4.png")
plt.show()

## Damped Newton without preconditioning

In [ ]:
rho = 0.95
c = 0.05
DampedNewtonP = []
results_DampedNewton  = []
times_DampedNewton    = []
Hessians_DampedNewton = []

for eps in epsilons:
    # Line Search
    print( "Damped Newton for epsilon = "+str(eps)+":" )    
    #Cost matrix
    C = p_norms(x,y,'inf')

    # a and b
    a = normalize( np.ones( N[0] ) )
    a = a.reshape( a.shape[0],-1 )
    b = normalize( np.ones( N[1] ) )
    b = b.reshape( b.shape[0],-1 )

    #Kernel
    K = np.exp(-C/eps)
    f,g = a,b

    print( "Doing for (",N[0],N[1],")." )
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.DampedNewton( K,a,b,f,g,eps,rho,c )
    out = Optimizer._update( maxiter=50 )
    results_DampedNewton.append( out )
    end = time.time()
    times_DampedNewton.append( end-start )
    print( " |- Computing P" )
    DampedNewtonP.append( GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)) )
    print( " |- Recording (unstabilized) Hessian \n" )

    mat  = -eps*Optimizer.Hessian
    diag = 1/np.sqrt( np.vstack( (a,b) ) ).flatten()
    mat = diag[:,None]*mat*diag[None,:]
    Hessians_DampedNewton.append( mat )
    

In [ ]:
eigs = []
eigvecs = []
for i in range( len(epsilons) ) :
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon="+str(eps) )
    ev = spectral_decomposition( Hessians_DampedNewton[i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")


In [ ]:
fig,ax = plt.subplots( figsize=(24,3),nrows=1, ncols=len(epsilons), sharey=True )
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
plt.subplots_adjust( wspace=0,hspace=0 )
plt.savefig("../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images/eigenhistunstabilizedL2.png")
plt.show()

## Damped Newton with preconditioning

In [ ]:
num_eigs = 10
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_DampedNewton[-1],N, ansatz=False )

In [ ]:
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians = []
DampedNewtonP = []
results_DampedNewton  = []
times_DampedNewton    = []

#epsilons = [ 0.3 ]
f, g = None, None
for eps in epsilons:
    # Line Search
    print( "Damped Newton for epsilon="+str(eps)+":" )    
    # Cost matrix
    C = p_norms(x,y,'inf')

    # a and b
    a = normalize( np.ones(N[0]) )
    a = a.reshape( a.shape[0],-1 )
    b = normalize( np.ones(N[1]) )
    b = b.reshape( b.shape[0],-1 )

    #Kernel
    K = np.exp(-C/eps)

    if (f is None) or (g is None): 
        f,g = a,b

    print( "Doing for (",N[0],N[1],")." )
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.DampedNewton_With_Preconditioner( K,a,b,f,g,eps,rho,c,null_vector,precond_vectors[:] )
    out = Optimizer._update( maxiter=50, iterative_inversion=30, version=None,debug=False,optType='cg' )
    results_DampedNewton.append( out )
    end = time.time()
    times_DampedNewton.append(end-start)
    print( " |- Computing P" )

    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
        # f = f.reshape( f.shape[0], -1)
        # g = g.reshape( g.shape[0], -1)
    
    DampedNewtonP.append( GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)) )
    final_modified_Hessians.append( Optimizer.modified_Hessian )


In [ ]:
plt.figure( figsize = (20,7) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range( len(results_DampedNewton) ):
  error = np.asarray( results_DampedNewton[i]['error_a'] )+np.asarray( results_DampedNewton[i]['error_b'] )
  plt.plot( error,label='Damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2 )

plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images/ErrorLinesearchNewton_final_cgL2.png" )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

# V. 1-cosine similarity

## Sinkhorn

In [ ]:
print("Sinkhorn.... ")
SinkhornP=[]
results_Sinkhorn=[]
times_Sinkhorn=[]
for eps in epsilons:

  
  #Cost matrix
  C =1- (x.transpose().dot(y))/(np.linalg.norm(x,2)*np.linalg.norm(y,2))
  
  # a and b
  a = normalize(np.ones(N[0]))
  a = a.reshape(a.shape[0],-1)
  b = normalize(np.ones(N[1]))
  b = b.reshape(b.shape[0],-1)



  #Kernel
  K = np.exp(-C/eps)


  print( "Doing for (",N[0],N[1],") and epsilon = " +str(eps)+ " ." )
  print( " |- Iterating" )

  #Inflating
  u = a
  v = b

  start = time.time()
  Optimizer = computational_OT.Sinkhorn( K,a,b,u,v,eps)
  out = Optimizer._update(maxiter=10000)
  results_Sinkhorn.append( out )
  end = time.time()
  times_Sinkhorn.append( end-start )
  print( " |- Computing P" )
  print( "" )
  SinkhornP.append(GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)))
  

In [ ]:

plt.figure( figsize = (20,7) )

plt.subplot(2,1,1),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len(results_Sinkhorn) ):
  error=np.asarray( results_Sinkhorn[i]['error_a'] )+np.asarray( results_Sinkhorn[i]['error_b'] )
  plt.plot( error,label='Sinkhorn for $\epsilon=$'+ str(epsilons[i]), linewidth = 2 )
plt.yscale( 'log' )
plt.legend()

plt.savefig("../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images/ConvergenceSinkhornvaryingepsilonL4.png")
plt.show()

## Damped Newton without preconditioning

In [ ]:
rho = 0.95
c = 0.05
DampedNewtonP = []
results_DampedNewton  = []
times_DampedNewton    = []
Hessians_DampedNewton = []

for eps in epsilons:
    # Line Search
    print( "Damped Newton for epsilon = "+str(eps)+":" )    
    #Cost matrix
    C =1- (x.transpose().dot(y))/(np.linalg.norm(x,2)*np.linalg.norm(y,2))

    # a and b
    a = normalize( np.ones( N[0] ) )
    a = a.reshape( a.shape[0],-1 )
    b = normalize( np.ones( N[1] ) )
    b = b.reshape( b.shape[0],-1 )

    #Kernel
    K = np.exp(-C/eps)
    f,g = a,b

    print( "Doing for (",N[0],N[1],")." )
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.DampedNewton( K,a,b,f,g,eps,rho,c )
    out = Optimizer._update( maxiter=50 )
    results_DampedNewton.append( out )
    end = time.time()
    times_DampedNewton.append( end-start )
    print( " |- Computing P" )
    DampedNewtonP.append( GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)) )
    print( " |- Recording (unstabilized) Hessian \n" )

    mat  = -eps*Optimizer.Hessian
    diag = 1/np.sqrt( np.vstack( (a,b) ) ).flatten()
    mat = diag[:,None]*mat*diag[None,:]
    Hessians_DampedNewton.append( mat )
    

In [ ]:
eigs = []
eigvecs = []
for i in range( len(epsilons) ) :
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon="+str(eps) )
    ev = spectral_decomposition( Hessians_DampedNewton[i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")


In [ ]:
fig,ax = plt.subplots( figsize=(24,3),nrows=1, ncols=len(epsilons), sharey=True )
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
plt.subplots_adjust( wspace=0,hspace=0 )
plt.savefig("../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images/eigenhistunstabilizedL2.png")
plt.show()

## Damped Newton with preconditioning

In [ ]:
num_eigs = 10
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_DampedNewton[-1],N, ansatz=False )

In [ ]:
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians = []
DampedNewtonP = []
results_DampedNewton  = []
times_DampedNewton    = []

#epsilons = [ 0.3 ]
f, g = None, None
for eps in epsilons:
    # Line Search
    print( "Damped Newton for epsilon="+str(eps)+":" )    
    # Cost matrix
    C =1- (x.transpose().dot(y))/(np.linalg.norm(x,2)*np.linalg.norm(y,2))

    # a and b
    a = normalize( np.ones(N[0]) )
    a = a.reshape( a.shape[0],-1 )
    b = normalize( np.ones(N[1]) )
    b = b.reshape( b.shape[0],-1 )

    #Kernel
    K = np.exp(-C/eps)

    if (f is None) or (g is None): 
        f,g = a,b

    print( "Doing for (",N[0],N[1],")." )
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.DampedNewton_With_Preconditioner( K,a,b,f,g,eps,rho,c,null_vector,precond_vectors[:] )
    out = Optimizer._update( maxiter=50, iterative_inversion=30, version=None,debug=False,optType='cg' )
    results_DampedNewton.append( out )
    end = time.time()
    times_DampedNewton.append(end-start)
    print( " |- Computing P" )

    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
        # f = f.reshape( f.shape[0], -1)
        # g = g.reshape( g.shape[0], -1)
    
    DampedNewtonP.append( GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)) )
    final_modified_Hessians.append( Optimizer.modified_Hessian )


In [ ]:
plt.figure( figsize = (20,7) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range( len(results_DampedNewton) ):
  error = np.asarray( results_DampedNewton[i]['error_a'] )+np.asarray( results_DampedNewton[i]['error_b'] )
  plt.plot( error,label='Damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2 )

plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonVsSinkhorn_benchmark_annulusVSsquare_images/ErrorLinesearchNewton_final_cgL2.png" )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )